### 0. 필요한 라이브러리 로드

In [1]:
import pymysql
from sqlalchemy import create_engine
# pip install mysqlclient

from abc import ABC, abstractmethod
from typing import *
import re

from prettytable import PrettyTable
import pandas as pd
pd.options.display.float_format = '{:.10f}'.format
import pprint

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 기존 클래스 (테이블 넣고 ~ 결합키, 결합대상정보 ~ 매핑테이블 만들기)

In [2]:
# import pymysql
# from abc import ABC, abstractmethod

class PreprocessQuery(ABC):
    """가명처리를 위한 개인정보 추출 목적의 SQL쿼리 추상클래스"""
    @abstractmethod
    def connectDatabase(self):
        """데이터베이스에 연결하기 위해 접속하는 메서드"""
        pass
    
    @abstractmethod
    def executeQuery(self, SQL):
        """SQL쿼리를 실행하는 메서드"""
        pass
    
    @abstractmethod
    def closeConnection(self):
        """데이터베이스와의 연결을 종료하는 메서드"""
        pass

In [3]:
# from pseudonymizer.encryptionPseudonyms.abstractpreprocessQuery import PreprocessQuery
# from typing import *

class PyMySQLQuery(PreprocessQuery):
    """MySQL Server데이터베이스에 연결하고 쿼리를 실행하여 데이터를 추출하는 클래스"""
    def __init__(self, pw):
        self._pw = pw
        self.DBconnection = ConnectMySQLserver(self._pw)
        self.SQL = None
    
    def connectDatabase(self, serverIP: str, port_num: int, user_name: str, database_name: str, kr_encoder: str):
        """MySQL DBMS 데이터베이스에 접속하는 메서드"""
        self.DBconnection.connectDatabase(serverIP, port_num, user_name, database_name, kr_encoder)
    
    def dataQueryLanguage(self, sql):
        """SQL쿼리문 작성 메서드(데이터 추출 쿼리문 캡슐화)"""
        self.SQL = f"{sql}"
    
    def executeQuery(self):
        """SQL쿼리문 실행 및 예외처리 메서드(데이터베이스로 쿼리를 보내서 실행)"""
        try:
            action_output = self.DBconnection.cursor.execute(self.SQL)
            return action_output
        except pymysql.Error as e:
            print(f"Error Executing Query: {e}")

    def useFetchallQuery(self):
        """SQL 쿼리 실행 결과의 cursor.fetchall() 을 사용할 수 있도록 하는 메서드"""
        try:
            action_output = self.DBconnection.cursor.execute(self.SQL)
            records = self.DBconnection.cursor.fetchall()
            return records
        except pymysql.Error as e:
            print(f"Executing query error: {e}")

    def executePatch(self, contents):
        """cursor.executemany() 를 사용할 수 있도록 하는 메서드"""
        try:
            print(self.SQL, contents)
            action_output = self.DBconnection.cursor.executemany(self.SQL, contents)
            return action_output
        except pymysql.Error as e:
            print(f"Error Executing Query: {e}")
            
    
    def commitTransaction(self):
        """실행결과를 확정(트랜잭션을 커밋)하는 메서드"""
        self.DBconnection.connection.commit()
    
    def closeConnection(self):
        """데이터베이스와의 연결을 종료하는 메서드"""
        self.DBconnection.close_connection()
        
    def executeQueryAsDataFrame(self):
        """SQL 쿼리를 실행한 결과를 판다스 데이터프레임으로 출력하는 메서드"""
        try:
            action_output = self.DBconnection.cursor.execute(self.SQL)
            records = self.DBconnection.cursor.fetchall()
            attributes = [i[0] for i in self.DBconnection.cursor.description]
            querydata = pd.DataFrame(records, columns = attributes)
            return querydata
        
        except pymysql.Error as e:
            print(f"Executing query error: {e}")

In [4]:
class ConnectMySQLserver:
    """데이터베이스 엔진을 연동하기 위한 접속 및 연결 종료 클래스"""
    def __init__(self, pw):
        self._pw = pw
        self.connection = None
        self.cursor = None
    
    def connectDatabase(self, serverIP: str, port_num: int, user_name: str, database_name: str, kr_encoder: str):
        """MySQL DBMS 데이터베이스에 접속 메서드
        : 서버IP주소, 사용자명, 계정 암호, 데이터베이스명, 한글 인코딩 방식"""
        try:
            self.connection = pymysql.connect(
                host=serverIP, port=port_num,
                user=user_name, password=self._pw,
                db=database_name, charset=kr_encoder
            )
            self.cursor = self.connection.cursor()
        except pymysql.Error as e:
            print(f"Error Connecting to MySQL from Python: {e}")
    
    def closeConnection(self):
        """연결 및 커서 닫기 메서드"""
        if self.cursor:
            self.cursor.close()
        if self.connection:
            self.connection.close()

In [5]:
import os
import binascii

class CreateKeyCol(PyMySQLQuery):
    """SALT값을 더해 결합키를 암호화하는 클래스"""
    def __init__(self, pw: str, serverIP: str, port_num: int, user_name: str, database_name: str, kr_encoder: str):
        self.pw = pw
        self.kr_encoder = kr_encoder
        super().__init__(pw = self.pw)
        super().connectDatabase(serverIP, port_num, user_name, database_name, kr_encoder)

    def createKey(self, func: str, schema: str, table: str, key_col: str, salt_col: str):
        """결합키 암호화 방식을 선택하여 실행시키는 메서드"""
        if func == "SHA256":
            self.applySHA256(schema, table, salt_col, key_col)
        elif func == "SHA512":
            self.applySHA512(schema, table, salt_col, key_col)

    def createSalt(self, schema: str, table: str, salt_col: str, key_col: str):
        """SALT값을 만들어 테이블 특정 컬럼에 붙이는 메서드"""
        # SALT값 컬럼 만들기
        super().dataQueryLanguage(f"ALTER TABLE {salt_col} VARCHAR(1000)")
        super().executeQuery()

        # SALT값을 만들고 컬럼에 입력하기
        super().dataQueryLanguage(f"SELECT * FROM {schema}.{table}")
        rows = super().useFetchallQuery()

        for row in rows:
            salt = binascii.hexlify(os.urandom(16)).decode(self.kr_encoder)
            sql = f"UPDATE {schema}.{table} SET {salt_col} = {salt} WHERE {key_col} = {row[0]}"
            super().dataQueryLanguage(sql)
            super().executeQuery()

            
    def applySHA256(self, schema: str, table: str, key_col: str, salt_col: str):
        """SHA256 해시함수를 통해 결합키 컬럼을 암호화하는 메서드"""
        sql = f"UPDATE {schema}.{table} SET {key_col} = SHA2(CONCAT({key_col}, {salt_col}), 256)"
        super().dataQueryLanguage(sql)
        super().executeQuery()

    def applySHA512(self, schema: str, table: str, key_col: str, salt_col: str):
        """SHA512 해시함수를 통해 결합키 컬럼을 암호화하는 메서드"""
        sql = f"UPDATE {schema}.{table} SET {key_col} = SHA2(CONCAT({key_col}, {salt_col}), 512)"
        super().dataQueryLanguage(sql)
        super().executeQuery()


In [20]:
class CreateMappingTable(JoinMySQL):
    """결합키연계정보, 일련번호, 결합키, SALT 등 매핑테이블 만드는 클래스"""
    def __init__(self, pw: str, serverIP: str, port_num: int, user_name: str, database_name: str, kr_encoder: str):
        self.pw = pw
        self._createKeyCol = None
        super().__init__(pw = self.pw)
        super().connectDatabase(serverIP, port_num, user_name, database_name, kr_encoder)

    def addCreateKeyCol(self, createKeyCol: CreateKeyCol):
        self._createKeyCol = createKeyCol

    def createCryptoKey(self, schemas: list, tables: list, func: str, salt_col: str, key_col: str):
        """SALT값을 만든 뒤 결합키를 암호화하는 메서드"""
        for i, table in enumerate(tables):
            self._createKeyCol.createSalt(schemas[i], table, salt_col, key_col)
            self._createKeyCol.createKey(schemas[i], func, table, salt_col, key_col)

    def renameCol(self, schemas: list, tables: list, serial_col: str, suffixes: list):
        """테이블별로 serial_col 이었던 컬럼명을 {serial_col}_{suffixes[i]} 형태로 바꾸는 메서드"""
        for i in range(len(tables)):
            super().dataQueryLanguage(f"ALTER TABLE {schemas[i]}.{tables[i]} RENAME COLUMN {serial_col} to {serial_col}_{suffixes[i]}")
            super().executeQuery()

    def joinDB(self, schemas: list, tables: list, serial_col: str, suffixes: list, result_schema: str, result_table: str, key_col: str):
        """일련번호를 결합키 기준으로 결합하여 매핑테이블 만드는 메서드

           예시 구문 : 
           CREATE table joined_view2 AS
           SELECT serialnum_2, serialnum_3, new_table_concat_r.result 
           FROM new_table_concat_r
           INNER JOIN new_table_concat_2 ON new_table_concat_r.result = new_table_concat_2.result;
           
           
        """
        super().dataQueryLanguage(f"DROP TABLE IF EXISTS {result_schema}.{result_table}")
        super().executeQuery()

        create_sql = f"CREATE TABLE {result_schema}.{result_table} AS "
        select_sql = f"SELECT {schemas[0]}.{tables[0]}.{key_col}, {schemas[0]}.{tables[0]}.{serial_col}_{suffixes[0]}, {schemas[1]}.{tables[1]}.{serial_col}_{suffixes[1]}  "
        from_sql = f"FROM {schemas[0]}.{tables[0]} "
        join_sql = f"INNER JOIN {schemas[1]}.{tables[1]} ON {schemas[0]}.{tables[0]}.{key_col} = {schemas[1]}.{tables[1]}.{key_col} "

        if len(tables) > 2:
            for i in range(2, len(tables)):
                select_sql += f"{serial_col}_{suffixes[i]}, "
                join_sql += f"INNER JOIN {schemas[i]}.{tables[i]} ON {schemas[0]}.{tables[0]}.{key_col} = {schemas[i]}.{tables[i]}.{key_col} "
        else:
            pass

        select_sql = select_sql[:-2] + " "

        sql = create_sql + select_sql + from_sql + join_sql

        super().dataQueryLanguage(sql)
        super().executeQuery()

NameError: name 'JoinMySQL' is not defined

In [12]:
from abc import ABC, abstractmethod

## 새 클래스 과정 1 : 원본 테이블을 결합키와 결합대상정보로 분할하여 매핑테이블 만들기
- 스키마-테이블 저장 및 조회 추상화 
    - 각 클래스에서는 인스턴스 변수로 저장하되, 이 스키마-테이블 구조를 내부에서 받아쓰는 실행 클래스는 스키마와 테이블을 클래스 변수로 모은 뒤 묶어서 사용하기

### 테이블 저장 클래스

In [5]:
# from abc import ABC, abstractmethod

class SQLTable(ABC):
    """스키마와 테이블로 이루어진 SQL 테이블명 구조를 저장하고 꺼내쓰는 목적의 추상 클래스"""
    @abstractmethod
    def setSchemaTable(self):
        """스키마명, 테이블명 저장 메서드"""
        pass

    @abstractmethod
    def getSchema(self):
        """스키마명 꺼내는 메서드"""
        pass

    @abstractmethod
    def getTable(self):
        """테이블명 꺼내는 메서드"""
        pass

In [6]:
class TableContainer(SQLTable):
    """스키마, 테이블 입출력 클래스"""
    def __init__(self):
        self._schema = None
        self._table = None

    def setSchemaTable(self, schema: str, table: str):
        """스키마, 테이블 입력 클래스"""
        self._schema = schema
        self._table = table

    def getSchema(self):
        return self._schema
    
    def getTable(self):
        return self._table



In [7]:
class InitTables:
    """원본 스키마&테이블, 결합키 스키마&테이블, 결합대상정보 스키마&테이블 모으기"""
    def __init__(self):
        self.original_table = None
        self.key_table = None
        self.target_table = None
        self.serial_col = None
        self.serial_text = None
        self.key_cols = None
        self.join_key = None

    def addOriginalTable(self, original_table: TableContainer):
        """원본 스키마 & 테이블 입력 메서드"""
        self.original_table = original_table

    def addKeyTable(self, key_table: TableContainer):
        """결합키 스키마 & 테이블 입력 메서드"""
        self.key_table = key_table

    def addTargetTable(self, target_table: TableContainer):
        """결합대상정보 스키마 & 테이블 입력 메서드"""
        self.target_table = target_table

    def addSerialColInfo(self, serial_col: str, serial_text: str, join_key: str):
        """일련번호 컬럼명 (serial_col), 일련번호 내용 (serial_text1, serial_text2 형태) 입력 메서드"""
        self.serial_col = serial_col
        self.serial_text = serial_text
        self.join_key = join_key

    def addKeyColInfo(self, columns: list):
        """결합키 생성 항목 컬럼명 입력 메서드"""
        self.key_cols = columns


In [8]:
class BundleTables(ABC):
    """결합키 테이블 저장, 결합대상정보 테이블 저장 클래스를 위한 추상화 클래스"""
    @abstractmethod
    def addInitTables(self):
        """원본테이블, 결합키 생성 테이블, 결합대상정보 테이블로 이루어진 InitTables 클래스 받는 추상 메서드"""
        pass

    @abstractmethod
    def selectTables(self):
        """InitTables에서 해당 클래스의 성격에 맞는 테이블만 골라내어 저장하기"""
        pass

    @abstractmethod
    def getTableList(self):
        """클래스 성격에 맞는 테이블을 출력"""
        pass

    @abstractmethod
    def getSchemas(self):
        """스키마명만 모아서 출력"""
        pass

    @abstractmethod
    def getTables(self):
        """테이블명만 모아서 출력"""
        pass

In [9]:
class KeyTables(BundleTables):
    """결합키 생성 테이블 저장 클래스"""
    init_tables = []
    key_tables = []
    columns = None
    serial_cols = []
    key_col = None

    @classmethod
    def addInitTables(cls, tables: InitTables):
        """원본테이블, 결합키 생성 테이블, 결합대상정보 테이블로 이루어진 InitTables 클래스 받기"""
        cls.init_tables.append(tables)

    @classmethod
    def addKeyCol(cls, key_col: str):
        """각 테이블에서 결합키가 놓일 컬럼명 더하기"""
        cls.key_col = key_col

    @classmethod
    def addColumns(cls, columns: list):
        """결합키 생성 항목 컬럼명 입력 메서드"""
        cls.columns = columns

    @classmethod
    def selectTables(cls):
        """InitTables에서 key_table만 골라내어 저장하기"""
        for table in cls.init_tables:
            cls.key_tables.append(table.key_table)
            cls.serial_cols.append(f"{table.serial_col}_{table.serial_text}")

    @classmethod
    def getTableList(cls):
        """결합키 생성 테이블 출력"""
        return cls.key_tables
    
    @classmethod
    def getSchemas(cls):
        """스키마명만 모아서 출력"""
        schemas = []
        for table in cls.key_tables:
            schemas.append(table.getSchema())

        return schemas
    
    @classmethod
    def getTables(cls):
        """테이블명만 모아서 출력"""
        tables = []
        for table in cls.key_tables:
            tables.append(table.getTable())

    @classmethod
    def getSerialCols(cls):
        return cls.serial_cols
    
    @classmethod
    def reset(cls):
        """클래스 변수 초기화"""
        cls.init_tables = []
        cls.key_tables = []
        cls.columns = None
        cls.serial_cols = []
        cls.key_col = None
    

In [10]:
class TargetTables(BundleTables):
    """결합대상정보 생성 테이블 저장 클래스"""
    init_tables = []
    target_tables = []
    columns = None
    serial_cols = []

    @classmethod
    def addInitTables(cls, tables: InitTables):
        """원본테이블, 결합키 생성 테이블, 결합대상정보 테이블로 이루어진 InitTables 클래스 받기"""
        cls.init_tables.append(tables)

    @classmethod
    def addColumns(cls, columns: list):
        """결합키 생성 항목 컬럼명 입력 메서드"""
        cls.columns = columns


    @classmethod
    def selectTables(cls):
        """InitTables에서 target_table만 골라내어 저장하기"""
        for table in cls.init_tables:
            cls.target_tables.append(table.target_table)
            cls.serial_cols.append(table.serial_col)


    @classmethod
    def getTableList(cls):
        """결합대상정보 생성 테이블 출력"""
        return cls.target_tables
    
    @classmethod
    def getSchemas(cls):
        """스키마명만 모아서 출력"""
        schemas = []
        for table in cls.target_tables:
            schemas.append(table.getSchema())

        return schemas
    
    @classmethod
    def getTables(cls):
        """테이블명만 모아서 출력"""
        tables = []
        for table in cls.target_tables:
            tables.append(table.getTable())

    @classmethod
    def reset(cls):
        cls.init_tables = []
        cls.target_tables = []
        cls.columns = None
        cls.serial_cols = []

### SQL 쿼리 실행 클래스

In [26]:
class DivideOriginalTable(PyMySQLQuery):
    """원본 테이블을 결합키와 결합대상정보로 분할하기"""
    def __init__(self, pw: str, serverIP: str, port_num: int, user_name: str, database_name: str, kr_encoder: str):
        super().__init__(pw = pw)
        super().connectDatabase(serverIP, port_num, user_name, database_name, kr_encoder)
        self.init_tables = None
        self.original_table = None
        self.serial_col = None
        self.serial_text = None
        self.key_cols = None

    def addInitTables(self, init_tables: InitTables):
        """원본 테이블, 결합키 테이블, 결합대상정보 테이블 객체 통해 입력"""
        self.init_tables = init_tables

        self.original_table = self.init_tables.original_table
        self.serial_col = self.init_tables.serial_col
        self.serial_text = self.init_tables.serial_text

        self.key_cols = self.init_tables.key_cols

    def addSerialNum(self):
        """일련번호 컬럼 추가 메서드"""
        schema = self.original_table.getSchema()
        table = self.original_table.getTable()

        # 컬럼명 중복시 해당 컬럼 삭제
        super().dataQueryLanguage(f"ALTER TABLE {schema}.{table} DROP COLUMN {self.serial_col}_{self.serial_text}")
        super().executeQuery()

        # 컬럼 만들기
        make_column = f"ALTER TABLE {schema}.{table} ADD COLUMN {self.serial_col}_{self.serial_text} VARCHAR(1000)"
        super().dataQueryLanguage(make_column)
        super().executeQuery()

        # 값 할당
        super().dataQueryLanguage("SET @counter = 0;")
        super().executeQuery()
        
        super().dataQueryLanguage(f"UPDATE {schema}.{table} SET {self.serial_col}_{self.serial_text} = CONCAT('{self.serial_text}', @counter := @counter + 1);")
        super().executeQuery()
        super().commitTransaction()

    def insertKey(self, salt: str):
        """결합키 테이블 DB 입력 메서드"""
        key_table = self.init_tables.key_table
        key_schema = key_table.getSchema()
        key_result = key_table.getTable()
        join_key = self.init_tables.join_key

        original_schema = self.original_table.getSchema()
        original_table = self.original_table.getTable()

        join_columns = ', '.join(self.key_cols)

        super().dataQueryLanguage(f"DROP TABLE IF EXISTS {key_schema}.{key_result}")
        super().executeQuery()
        
        sql = f"CREATE TABLE {key_schema}.{key_result} AS SELECT {self.serial_col}_{self.serial_text}, {join_columns} FROM {original_schema}.{original_table}"
        super().dataQueryLanguage(sql)
        super().executeQuery()

        # 결합키 컬럼 만들기
        super().dataQueryLanguage(f"ALTER TABLE {key_schema}.{key_result} ADD COLUMN {join_key} VARCHAR(1000)")
        super().executeQuery()

        super().dataQueryLanguage(f"UPDATE {key_schema}.{key_result} SET {join_key} = CONCAT({join_columns})")
        super().executeQuery()

        # SALT값 컬럼 만들고 채우기
        super().dataQueryLanguage(f"ALTER TABLE {key_schema}.{key_result} ADD COLUMN SALT VARCHAR(1000)")
        super().executeQuery()

        super().dataQueryLanguage(f"UPDATE {key_schema}.{key_result} SET SALT = '{salt}'")
        super().executeQuery()

        super().commitTransaction()
        
    def insertTarget(self):
        """결합대상정보 테이블 DB 입력 메서드
           : 원래 테이블을 복사한 뒤, 결합키 생성항목 columns를 제거하여 생성
        """
        target_table = self.init_tables.target_table
        target_schema = target_table.getSchema()
        target_result = target_table.getTable()

        original_schema = self.original_table.getSchema()
        original_table = self.original_table.getTable()

        super().dataQueryLanguage(f"DROP TABLE IF EXISTS {target_schema}.{target_result}")
        super().executeQuery()

        create_sql = f"CREATE TABLE {target_schema}.{target_result} AS SELECT * FROM {original_schema}.{original_table}"
        super().dataQueryLanguage(create_sql)
        super().executeQuery()

        for column in self.key_cols:
            drop_sql = f"ALTER TABLE {target_schema}.{target_result} DROP COLUMN {column}"
            super().dataQueryLanguage(drop_sql)
            super().executeQuery()

        super().commitTransaction()


In [18]:
class EncryptKeyCol(PyMySQLQuery):
    """결합키 암호화 클래스 : InitTables에 들어있는 key_table의 join_key에 적용"""
    def __init__(self, pw: str, serverIP: str, port_num: int, user_name: str, database_name: str, kr_encoder: str):
        self.kr_encoder = kr_encoder
        super().__init__(pw = pw)
        super().connectDatabase(serverIP, port_num, user_name, database_name, kr_encoder)
        
        self.init_tables = None
        self.key_tables = None
        self.salt_col = None

    def addInitTables(self, init_tables: InitTables):
        """원본 테이블, 결합키 테이블, 결합대상정보 테이블 객체 통해 입력"""
        self.init_tables = init_tables
        self.serial_col = self.init_tables.serial_col
        self.serial_text = self.init_tables.serial_text

    def addSaltCol(self, salt_col: str):
        """Salt키 컬럼명 입력"""
        self.salt_col = salt_col

    def encryptKeyCol(self, func: str):
        """self.init_tables에 저장된 모든 key_table을 대상으로 암호화를 실행하는 메서드"""
        schema = self.init_tables.key_table.getSchema()
        table = self.init_tables.key_table.getTable()
        serial_col = f"{self.serial_col}_{self.serial_text}"
        self.createKey(func, schema, table, self.init_tables.join_key, self.salt_col)

    def createKey(self, func: str, schema: str, table: str, key_col: str, salt_col: str):
        """결합키 암호화 방식을 선택하여 실행시키는 메서드"""
        if func == "SHA256":
            self.applySHA256(schema, table, key_col, salt_col)
        elif func == "SHA512":
            self.applySHA512(schema, table, key_col, salt_col)
        else:
            print("SHA256과 SHA512 중 하나를 입력하십시오")

    def applySHA256(self, schema: str, table: str, key_col: str, salt_col: str):
        """SHA256 해시함수를 통해 결합키 컬럼을 암호화하는 메서드"""
        sql = f"UPDATE {schema}.{table} SET {key_col} = SHA2(CONCAT({key_col}, {salt_col}), 256)"
        super().dataQueryLanguage(sql)
        super().executeQuery()
        super().commitTransaction()

    def applySHA512(self, schema: str, table: str, key_col: str, salt_col: str):
        """SHA512 해시함수를 통해 결합키 컬럼을 암호화하는 메서드"""
        sql = f"UPDATE {schema}.{table} SET {key_col} = SHA2(CONCAT({key_col}, {salt_col}), 512)"
        super().dataQueryLanguage(sql)
        super().executeQuery()
        super().commitTransaction()


In [13]:
class CreateMappingTable(PyMySQLQuery):
    """매핑테이블 만들기 클래스"""
    def __init__(self, pw: str, serverIP: str, port_num: int, user_name: str, database_name: str, kr_encoder: str):
        super().__init__(pw = pw)
        super().connectDatabase(serverIP, port_num, user_name, database_name, kr_encoder)

        self.key_tables = None
        self.mapping_table = None
        
    def addKeyTables(self, key_tables: KeyTables):
        """결합키 테이블 객체 입력"""
        self.key_tables = key_tables

    def addMappingTable(self, mapping_table: TableContainer):
        """매핑테이블 스키마, 테이블 이름 입력"""
        self.mapping_table = mapping_table

    def joinDB(self):
        """일련번호를 결합키 기준으로 결합하여 매핑테이블 만드는 메서드
           
           예시 구문 : 
           CREATE table joined_view2 AS
           SELECT serialnum_2, serialnum_3, new_table_concat_r.result 
           FROM new_table_concat_r
           INNER JOIN new_table_concat_2 ON new_table_concat_r.result = new_table_concat_2.result;
        """
        result_schema = self.mapping_table.getSchema()
        result_table = self.mapping_table.getTable()

        schemas = self.key_tables.getSchemas()
        tables = self.key_tables.getTables()
        serial_cols = self.key_tables.getSerialCols()

        super().dataQueryLanguage(f"DROP TABLE IF EXISTS {result_schema}.{result_table}")
        super().executeQuery()

        create_sql = f"CREATE TABLE {result_schema}.{result_table} AS "
        select_sql = f"SELECT {schemas[0]}.{tables[0]}.{self.key_tables.key_col}, {schemas[0]}.{tables[0]}.{serial_cols[0]}, {schemas[1]}.{tables[1]}.{serial_cols[1]}  "
        from_sql = f"FROM {schemas[0]}.{tables[0]} "
        join_sql = f"INNER JOIN {schemas[1]}.{tables[1]} ON {schemas[0]}.{tables[0]}.{self.key_tables.key_col} = {schemas[1]}.{tables[1]}.{self.key_tables.key_col} "

        if len(tables) > 2:
            for i in range(2, len(tables)):
                select_sql += f"{serial_cols[i]}, "
                join_sql += f"INNER JOIN {schemas[i]}.{tables[i]} ON {schemas[0]}.{tables[0]}.{self.key_tables.key_col} = {schemas[i]}.{tables[i]}.{self.key_tables.key_col} "
        else:
            pass

        select_sql = select_sql[:-2] + " "

        sql = create_sql + select_sql + from_sql + join_sql

        super().dataQueryLanguage(sql)
        super().executeQuery()

In [14]:
class JoinTargetData(PyMySQLQuery):
    """매핑테이블의 일련번호를 기준으로 결합대상정보를 결합하는 클래스"""
    def __init__(self, pw: str, serverIP: str, port_num: int, user_name: str, database_name: str, kr_encoder: str):
        super().__init__(pw = pw)
        super().connectDatabase(serverIP, port_num, user_name, database_name, kr_encoder)

        self.target_tables = None
        self.mapping_table = None
        self.result = None

    def addTargetTables(self, target_tables: TargetTables):
        """결합대상정보 테이블 객체 입력"""
        self.target_tables = target_tables

    def addMappingTable(self, mapping_table: TableContainer):
        """매핑테이블 스키마, 테이블 이름 입력"""
        self.mapping_table = mapping_table

    def addResult(self, result: TableContainer):
        """결합대상정보 결합 클래스 내용 주입"""
        self.result = result

    def joinDB(self):
        """매핑테이블의 일련번호를 기준으로 결합대상정보를 결합하는 메서드
           결합키를 제외한 컬럼명들을 SELECT에 나열하고, INNER JOIN을 한줄씩 더하기
        """
        schemas = self.target_tables.getSchemas()
        tables = self.target_tables.getTables()
        serial_cols = self.target_tables.serial_cols
        mapping_schema = self.mapping_table.getSchema()
        mapping_table = self.mapping_table.getTable()
        result_schema = self.result.getSchema()
        result_table = self.result.getTable()

        create_sql = f"CREATE TABLE v{result_schema}.{result_table} AS "
        select_sql = f"SELECT {mapping_schema}.{mapping_table}.*, "
        from_sql = f"FROM {mapping_schema}.{mapping_table} "
        join_sql = f""

    
        for i in range(len(tables)):
            # 컬럼명 전부 구하고 그중 매핑테이블 컬럼명 빼서 SELECT 구문에 포함시키기
            super().dataQueryLanguage(f"SELECT column_name FROM information_schema.columns WHERE table_schema = '{result_schema}' AND table_name = '{tables[i]}'")
            result = super().executeQueryAsDataFrame()

            li = result['COLUMN_NAME'].tolist()
            li.remove(f"{serial_cols[i]}")
            select_sql += f"{', '.join(li)}, "
            
            join_sql += f"INNER JOIN {schemas[i]}.{tables[i]} ON {schemas[i]}.{tables[i]}.{serial_cols[i]} = {mapping_table}.{serial_cols[i]}"
            
        select_sql = select_sql[:-2] + " "

        sql = create_sql + select_sql + from_sql + join_sql

        super().dataQueryLanguage(sql)
        super().executeQuery()

## 최상위 실행 클래스
- 각 원본 테이블별 결합키 테이블, 결합대상정보 테이블 분할 정보 입력
- 결합키 테이블 각각 결합키 암호화 진행
- 결합키 테이블을 암호화 기준으로 묶어 매핑테이블 만들기
- 매핑테이블 일련번호 기준 결합대상정보 결합
- 결합대상정보와 원본 테이블 합쳐 반출

- 처음에는 keytables, targettables 나눠서 했는데 이거 어차피 리스트 단위로 테이블 관리하면 망함
- inittables에서 keytable 각각 처리하는식으로 가야함

In [ ]:
class CryptoGrapher:
    """원본 테이블 입력부터 반출까지 전부 실행하는 최상위 실행 클래스"""
    def __init__(self):
        pass

    

## 테스트 케이스
#### 테스트 케이스 1 : DATA_JOIN_CARDPAYMENT + DATA_JOIN_ACCOMODATIONAPP
1. DATA_JOIN_CARDPAYMENT 와 DATA_JOIN_ACCOMODATIONAPP 테이블의 일련번호를 각 테이블의 SERIALNUM 컬럼에서 C1~C10000 형태, A1~A10000 형태로 매기기
2. DATA_JOIN_CARDPAYMENT 와  테이블의 결합키대상정보를 CARDKEY와 APPKEY 테이블, 결합대상정보를 CARDTARGET, APPTARGET 테이블에 저장
3. CARDKEY와 APPKEY 테이블에서 결합키대상정보 컬럼 3개의 내용을 묶어서 JOINKEY 컬럼에 저장
4. CARDKEY와 APPKEY 테이블에서 일련번호를 저장하는 SERIALNUM 컬럼명을 각각 SERIALNUM_C, SERIALNUM_A 으로 변경
5. 결합키 JOINKEY 컬럼을 기준으로 CARDKEY와 APPKEY 테이블의 일련번호를 결합하여 매핑테이블 CARD_APP_MAP 생성
6. 매핑테이블의 각 일련번호 컬럼을 기준으로 CARDTARGET과 APPTARGET 테이블을 결합하여 JOINED_TARGET_CARD_APP 생성
7. JOINED_TARGET_CARD_APP 테이블에서 AFFILIGATESTORE_ADDRESS, AFFILIGATESTORE_INDUSTRY_CODE, USE_SERVICE_AREA, USE_SERVICE_DATE 컬럼을 뽑아 FINAL_RESULT 테이블에 저장. 이 과정에서 DATA_JOIN_ACCOMODATIONAPP 테이블과 비교하여 결합률을 선정하고 반출여부 심사.

In [23]:
accomo = TableContainer()
accomo.setSchemaTable(schema="FINANCIALCONSUMER", table="DATA_JOIN_ACCOMODATIONAPP")
card = TableContainer()
card.setSchemaTable(schema="FINANCIALCONSUMER", table="DATA_JOIN_CARDPAYMENT")

In [24]:
accomo_init = InitTables()
accomo_init.addOriginalTable(original_table=accomo)
accomo_key = TableContainer()
accomo_key.setSchemaTable(schema="FINANCIALCONSUMER", table="APPKEY")
accomo_init.addKeyTable(key_table=accomo_key)
accomo_target = TableContainer()
accomo_target.setSchemaTable(schema="FINANCIALCONSUMER", table="APPTARGET")
accomo_init.addTargetTable(target_table=accomo_target)
accomo_init.addKeyColInfo(columns=['NAME', 'BIRTH_DATE', 'GENDER'])
accomo_init.addSerialColInfo(serial_col="SERIALNUM", serial_text="A", join_key="JOINKEY")

card_init = InitTables()
card_init.addOriginalTable(original_table=card)
card_key = TableContainer()
card_key.setSchemaTable(schema="FINANCIALCONSUMER", table="CARDKEY")
card_init.addKeyTable(key_table=card_key)
card_target = TableContainer()
card_target.setSchemaTable(schema="FINANCIALCONSUMER", table="CARDTARGET")
card_init.addTargetTable(target_table=card_target)
card_init.addKeyColInfo(columns=['NAME', 'BIRTH_DATE', 'GENDER'])
card_init.addSerialColInfo(serial_col="SERIALNUM", serial_text="C", join_key="JOINKEY")

In [27]:
dv1 = DivideOriginalTable(pw = "1234", serverIP = "localhost", port_num = 3306, user_name = "root", database_name = "FINANCIALCONSUMER", kr_encoder = "utf8")
dv1.addInitTables(init_tables=accomo_init)
dv1.addSerialNum()
dv1.insertKey(salt='703711bfb24c7bcc0970c0836071e547')
dv1.insertTarget()

In [19]:
dv2 = DivideOriginalTable(pw = "1234", serverIP = "localhost", port_num = 3306, user_name = "root", database_name = "FINANCIALCONSUMER", kr_encoder = "utf8")
dv2.addInitTables(init_tables=card_init)
dv2.addSerialNum()
dv2.insertKey()
dv2.insertTarget()

In [16]:
key_tables = KeyTables()
key_tables.addInitTables(tables=accomo_init)
key_tables.addInitTables(tables=card_init)
key_tables.addColumns(columns=['NAME', 'BIRTH_DATE', 'GENDER'])
key_tables.addKeyCol(key_col="JOINKEY")

In [28]:
key_tables.reset()

In [17]:
key_tables.selectTables()

In [18]:
key_tables.init_tables

In [19]:
eq = EncryptKeyCol(pw = "1234", serverIP = "localhost", port_num = 3306, user_name = "root", database_name = "FINANCIALCONSUMER", kr_encoder = "utf8")
# eq.addKeyTables(key_tables=key_tables)
eq.addInitTables(init_tables=accomo_init)
eq.addSaltCol(salt_col="SALT")
eq.encryptKeyCol(func="SHA256")
# 현재 1000줄 createSalt에 4분 36초 소요, 10000줄 createSalt는 
# commitTransaction 횟수 줄여서 3분 45초로 최적화
# 1000줄 createSalt + createKey = 4분 46초

FINANCIALCONSUMER APPKEY JOINKEY SALT SERIALNUM_A
UPDATE APPKEY SET SALT = %s WHERE SERIALNUM_A = %s [('703711bfb24c7bcc0970c0836071e547', 'A1'), ('743b81aaf864bb5710296eed8eb1e581', 'A2'), ('1fc43ef4f41fd12349201393f7392eb3', 'A3'), ('24401024a5df3e9931d15f09b80100d1', 'A4'), ('3c2fb0021b53813c769ee13b90325760', 'A5'), ('8f1b2306e962f35802e5449735b6519d', 'A6'), ('45290759b86f8966fe8c2ad2601ff512', 'A7'), ('42c69555b309cb498cee39fbdfa6dd35', 'A8'), ('8f8617a30219dd84f49addc5079a87f1', 'A9'), ('48008ee2d378b1ea10688ce2d438497f', 'A10'), ('bf812f4bd461ba9df4a6f55382eccc84', 'A11'), ('bcab591e4f1bb3262795b7c2275d8e03', 'A12'), ('36c4377e2e01d039ce54d051954fb7e6', 'A13'), ('e057871e0204d113949b634efb4a3720', 'A14'), ('bf57a25a8bd253332405e126bc566d03', 'A15'), ('716c63e8aea900555181c5d0bf6adedc', 'A16'), ('2c385ed2670a0637bd63ee562d3f8b01', 'A17'), ('9c495ddb6155bd6497c617ae755fc222', 'A18'), ('99d65491f884d9e8c72e0482e0ef01b4', 'A19'), ('83e2e61780e5c05d85cd6e760fb1e368', 'A20'), ('70cf5